# Imports

In [1]:
import os
import sys
import json
import xarray as xr
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt

In [2]:
# go up until we are in the project base directory
base_dir = os.getcwd()
while base_dir.split('/')[-1] != 'provide':
    base_dir = os.path.normpath(os.path.join(base_dir, '..'))

# add paths for tools and data
things_to_add = ['general_tools', 'aggregation_tools', 'general_data_for_aggregation']
for thing in things_to_add:
    sys.path.append(os.path.join(base_dir, thing))

# import stuff we need
from general_tools import check_if_notebook, mkdir
from oggm_result_filepath_and_realisations import (gcms_mesmer, quantiles_mesmer,
    scenarios_mesmer, oggm_result_dir, provide_regions, raw_oggm_output_file)
from aggregation_preprocessing import open_grid_from_dict
from actual_aggregation import open_files_and_aggregate_on_map, aggregating_scenario, check_slurm_done

In [3]:
# Use this to conditionally execute tests/debugging
if check_if_notebook():
    is_notebook = True
else:
    is_notebook = False

# Define directories

In [4]:
resolution_dir = 'total_data'

In [5]:
preprocess_region_dict_outpath = os.path.join(base_dir, 'global', resolution_dir)
mkdir(preprocess_region_dict_outpath);

# Open data

In [6]:
with open(os.path.join(preprocess_region_dict_outpath, "preprocessed_region_grids.json"), 'r') as f:
    region_structure_dict = json.load(f)

# Code for running on cluster

## define experiments for multiprocessing

In [7]:
# create dict for sbatch --array=
slurm_arrays = {}

count = 1
for region in region_structure_dict.keys():
    for scenario in scenarios_mesmer:
        slurm_arrays[str(count)] = (region, scenario)
        count += 1

if is_notebook:
    print(slurm_arrays)

{'1': ('global', 'CurPol'), '2': ('global', 'GS'), '3': ('global', 'LD'), '4': ('global', 'ModAct'), '5': ('global', 'Ref'), '6': ('global', 'Ren'), '7': ('global', 'Neg'), '8': ('global', 'SP'), '9': ('global', 'ssp119'), '10': ('global', 'ssp534-over')}


## run current experiment

In [ ]:
if not is_notebook:
    slurm_id = os.environ.get('ARRAY_ID', None)
    # convert slurm array to region and scenario
    region, scenario = slurm_arrays[slurm_id]

    # save results on cluster and copy at the end in run_slurm-file
    working_dir_cluster = os.environ.get('OGGM_WORKDIR', None)

    aggregated_data_outpath = os.path.join(
        working_dir_cluster, 'aggregated_data')
    mkdir(aggregated_data_outpath);

    aggregated_data_intermediate_outpath = os.path.join(
        preprocess_region_dict_outpath,
        'aggregated_data_intermediate')
    mkdir(aggregated_data_intermediate_outpath);

    aggregating_scenario(
            target_name=region,
            target_structure_dict=region_structure_dict,
            scenario=scenario,
            output_folder=aggregated_data_outpath,
            oggm_result_dir=oggm_result_dir,
            raw_oggm_output_file=raw_oggm_output_file,
            intermediate_data_folder=aggregated_data_intermediate_outpath,
            variables=['volume', 'area', 'thinning_rate', 'runoff'],
            risk_variables=['volume', 'area'],
            risk_thresholds=np.append(np.arange(10, 91, 10), [99]),  # in % melted of 2020, 10% means 10% of 2020 melted
            time_steps=np.arange(2015, 2101, 5),
            reset_files=False,
            add_map_data=False,
        )

## check which experiments failed for rerunning

In [4]:
if is_notebook:
    check_slurm_done(477614)

Files without 'SLURM DONE':
sbatch --array=6 run_slurm_aggregation_workflow.sh


In [11]:
if is_notebook:
    check_slurm_done(436338)

Files without 'SLURM DONE':
sbatch --array=342,237,212,303,320,186,349,300,59,322 run_slurm_aggregation_workflow.sh


In [12]:
if is_notebook:
    check_slurm_done(436392)

Files without 'SLURM DONE':
sbatch --array=59 run_slurm_aggregation_workflow.sh


In [14]:
if is_notebook:
    check_slurm_done(436405)

All files contain 'SLURM DONE'.


In [8]:
with xr.open_dataset('/home/www/pschmitt/provide/aggregate_data/github/provide/basins/total_data/aggregated_data_intermediate/rhine/total_data/rhine_CurPol_ACCESS-ESM1-5_0.05_total_data.nc') as ds:
    ds_rhine = ds

In [9]:
with xr.open_dataset('/home/www/pschmitt/provide/aggregate_data/github/provide/global/total_data/aggregated_data_intermediate/global/total_data/global_CurPol_ACCESS-ESM1-5_0.05_total_data.nc') as ds:
    ds_global = ds

In [10]:
ds_rhine

<xarray.Dataset>
Dimensions:         (gcm: 1, scenario: 1, quantile: 1, time: 17, lat: 7, lon: 8)
Coordinates:
  * gcm             (gcm) object 'ACCESS-ESM1-5'
  * scenario        (scenario) object 'CurPol'
  * quantile        (quantile) float64 0.05
  * time            (time) float64 2.02e+03 2.025e+03 ... 2.095e+03 2.1e+03
  * lat             (lat) float64 46.5 47.5 48.5 49.5 50.5 51.5 52.5
  * lon             (lon) float64 4.5 5.5 6.5 7.5 8.5 9.5 10.5 11.5
    hydro_year      (time) int64 ...
    hydro_month     (time) int64 ...
    calendar_year   (time) int64 ...
    calendar_month  (time) int64 ...
Data variables:
    volume          (gcm, scenario, quantile, time, lat, lon) float64 ...
    area            (gcm, scenario, quantile, time, lat, lon) float64 ...
    runoff          (gcm, scenario, quantile, time, lat, lon) float32 ...
    thinning_rate   (gcm, scenario, quantile, time, lat, lon) float64 ...